# Referenced Mosaic

# Setup

## Imports

In [ ]:
import os

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.utils import check_random_state
import yaml

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

In [ ]:
from night_horizons import utils, preprocess, mosaic, raster

## Settings

In [ ]:
with open('./config.yml', "r", encoding='UTF-8') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
local_settings = {
    'mosaic_filepath': 'mosaics/referenced.tiff',
    'overwrite': True,
    'random_state': 16849,
}
settings.update(local_settings)

## Parse Settings

In [ ]:
settings['mosaic_filepath'] = os.path.join(settings['data_dir'], settings['mosaic_filepath'])

In [ ]:
for key, relpath in settings['paths_relative_to_data_dir'].items():
    settings[key] = os.path.join(settings['data_dir'], relpath)

In [ ]:
random_state = check_random_state(settings['random_state'])

In [ ]:
referenced_fps = {
    i: utils.discover_data(
        settings['referenced_images_dir'],
        ['tif', 'tiff'],
        pattern=r'Geo\s\d+_' + f'{i}.tif'
    )
    for i in range(3)
}
referenced_fps = pd.concat(referenced_fps)

In [ ]:
assert len(referenced_fps) > 0

# Extract/Transform/Load

In [ ]:
preprocessing = preprocess.GeoTIFFPreprocesser()

In [ ]:
X = preprocessing.fit_transform(
    referenced_fps,
)

# Build Mosaic

In [ ]:
if os.path.isfile(settings['mosaic_filepath']) and settings['overwrite']:
    os.remove(settings['mosaic_filepath'])

In [ ]:
model = mosaic.ReferencedMosaic(settings['mosaic_filepath'])

In [ ]:
model.fit_transform(X)

# Evaluate

## Individual Inspection

In [ ]:
ind = random_state.choice(X.index)
row = X.loc[ind]

In [ ]:
original_img = utils.load_image(row['filepath'])
mosaic_img = model.get_image_with_bounds(row['x_min'], row['x_max'], row['y_min'], row['y_max'])

In [ ]:
subplot_mosaic = [['original', 'mosaic']]
n_rows = len(subplot_mosaic)
n_cols = len(subplot_mosaic[0])

fig = plt.figure(figsize=(10*n_cols, 10*n_rows))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['original']
ax.imshow(original_img)

ax = ax_dict['mosaic']
ax.imshow(mosaic_img)

## Overall Scores

In [ ]:
r_median = model.score(X)

In [ ]:
sns.histplot(model.scores_)

In [ ]:
sns.scatterplot(
    x=np.arange(len(model.scores_)),
    y=model.scores_,
)

TODO: For a proper comparison between the less-referenced and the referenced mosaics, we must aim to replicate the order at least partially.

## Calculate the area

In [ ]:
full_mosaic_img = model.get_image(0, 0, model.x_size_, model.y_size_)

In [ ]:
is_not_empty = ~np.isclose(full_mosaic_img, 0.)

In [ ]:
full_area = is_not_empty.sum() * -model.pixel_height_ * model.pixel_width_
full_area_km = full_area / (1000.)**2.
print(f'The area covered by the referenced mosaic is {full_area_km:.2g} km^2')

In [ ]:
# From google
actual_area_km = 953.

In [ ]:
# We only check if the area is large enough because we image the area around Indianapolis too
assert full_area_km > actual_area_km

# Close

In [ ]:
model.close()